# Sync

In [2]:
!aws s3 sync resources/grn-benchmark s3://openproblems-data/resources/grn/grn-benchmark --delete
!aws s3 sync resources/grn_models/ s3://openproblems-data/resources/grn/grn_models --delete
!aws s3 sync resources/prior/ s3://openproblems-data/resources/grn/prior --delete
# !aws s3 sync resources/supplementary/ s3://openproblems-data/resources/grn/supplementary --delete
# !aws s3 sync resources/results/ s3://openproblems-data/resources/grn/results --delete

/bin/bash: aws: command not found
/bin/bash: aws: command not found
/bin/bash: aws: command not found


In [ ]:
!cp resources/prior resources_test/prior

In [2]:
# !aws s3 sync resources_test/ s3://openproblems-data/resources_test/grn/ --delete

In [3]:
!aws s3 sync s3://openproblems-data/resources/grn/grn-benchmark resources/grn-benchmark 
# !aws s3 sync  s3://openproblems-data/resources/grn/grn_models resources/grn_models/
# !aws s3 sync s3://openproblems-data/resources/grn/prior resources/prior/ 


/bin/bash: aws: command not found


# Import

In [1]:
%reload_ext autoreload
%autoreload 2
import yaml
import os
import pandas as pd
import anndata as ad 
import numpy as np
import scanpy as sc 
from src.exp_analysis.helper import plot_cumulative_density
import matplotlib.pyplot as plt
import sys 
import subprocess
import io
import warnings

# Suppress all FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

sys.path.append('../')
from grn_benchmark.src.helper import surragate_names
from src.helper import *
par = {
    # 'methods': [ 'collectri', 'negative_control', 'positive_control', 'pearson_corr', 'portia', 'ppcor', 'genie3', 'grnboost2', 'scenic', 'scglue', 'celloracle'],
    'methods': [ 'collectri', 'negative_control', 'positive_control', 'pearson_corr', 'portia', 'ppcor', 'grnboost2', 'scenic', 'scglue', 'celloracle', 'scenicplus'],
    'models_dir': 'resources/grn_models/',
    'scores_dir': 'resources/scores'
}

ModuleNotFoundError: No module named 'lightgbm'

# Prior 

In [18]:
if False: 
    create_skeleton() # create tf2gene putative links


# Run grn inference 

In [27]:
if True: # local runs
    run_grn_inference()
if False: # r based methods
    !sbatch scripts/sbatch/ppcor.sh

scenicplus
Job scenicplus submitted successfully.
Submitted batch job 7765370



In [62]:
!ls resources/grn-benchmark/mccalla/inference

han.h5ad  jackson.h5ad	shalek.h5ad


# Calculate scores

In [47]:
if False: # consensus: run this after updating grns
    run_consensus(par)

if True: # run metrics/script_all.py
    calculate_scores()

Submitted batch job 7761215


In [2]:
!ls resources/scores/

10000-skeleton_False-binarize_False_lognorm-ridge.csv
10000-skeleton_False-binarize_False_pearson-ridge.csv
10000-skeleton_False-binarize_True_lognorm-ridge.csv
10000-skeleton_False-binarize_True_pearson-ridge.csv
10000-skeleton_True-binarize_False_lognorm-ridge.csv
10000-skeleton_True-binarize_False_pearson-ridge.csv
10000-skeleton_True-binarize_True_lognorm-ridge.csv
10000-skeleton_True-binarize_True_pearson-ridge.csv
50000-skeleton_False-binarize_False_lognorm-ridge.csv
50000-skeleton_False-binarize_False_pearson-ridge.csv
50000-skeleton_False-binarize_True_lognorm-ridge.csv
50000-skeleton_False-binarize_True_pearson-ridge.csv
50000-skeleton_True-binarize_False_lognorm-ridge.csv
50000-skeleton_True-binarize_False_pearson-ridge.csv
50000-skeleton_True-binarize_True_lognorm-ridge.csv
50000-skeleton_True-binarize_True_pearson-ridge.csv


In [4]:
df_scores = pd.read_csv(f"resources/scores/50000-skeleton_True-binarize_True_pearson-ridge.csv", index_col=0)
df_all_n = (df_scores-df_scores.min(axis=0))/(df_scores.max(axis=0)-df_scores.min(axis=0))
df_scores['rank'] = df_all_n.mean(axis=1).rank(ascending=False).astype(int)
df_scores.style.background_gradient()

,S1,S2,static-theta-0.0,static-theta-0.5,static-theta-1.0,rank
collectri,-0.052885,-0.162282,0.241939,0.311653,0.279391,13
negative_control,-0.038053,-0.053455,0.204835,0.298616,0.279320,12
positive_control,0.285482,0.497354,0.558247,0.437084,0.291442,3
pearson_corr,0.227900,0.418176,0.558176,0.433777,0.290445,4
portia,0.114365,0.247659,0.463710,0.342678,0.281718,7
ppcor,-0.009030,-0.026782,0.328378,0.315634,0.278937,10
grnboost2,0.277384,0.388048,0.583035,0.501260,0.305520,2
scenic,0.132473,0.197608,0.530981,0.448723,0.308812,6
granie,0.065566,0.088314,0.166634,0.254159,0.268779,11
scglue,0.054329,0.253396,0.477133,0.334458,0.282163,8


# Format resourcs used

In [19]:
if True: 
    # job_ids_dict_hvg = { 
    #     'portia': 7744548,
    #     'grnboost2': 7742249,
    #     'scenic': 7742283,
    #     'genie3': 7742285,
    #     'ppcor': 7742364,
    #     'scglue': 7742343,
    # }

    job_ids_dict = { 
        'portia': 7751292,
        'grnboost2': 7747906,
        'scenic': 7748219,
        'ppcor': 7748321,
        'scglue': [7756286, 7756675],
        'scenicplus': [7761874, 7760439, 7760554],
        'figr': 7756664,
        'celloracle': 7761872
    }
    
    df_res = process_trace_local(job_ids_dict)
    df_res = df_res[['Elapsed', 'MaxVMSize']]
    granie = pd.DataFrame({'Elapsed':  3643.337/60/60, 'MaxVMSize': 41},index=['granie'])
    df_res = pd.concat([df_res, granie], axis=0)
    df_res.columns = ['Duration (hour)', 'Peak memory (GB)']


## Merge scores with resources

In [20]:
df_scores = pd.read_csv(f"resources/scores/50000-skeleton_True-binarize_True_pearson-ridge.csv", index_col=0)

In [21]:
# create ranking 
df_scores = df_scores.fillna(0)
df_scores[df_scores < 0] = 0
df_scores = (df_scores-df_scores.min(axis=0))/(df_scores.max(axis=0)-df_scores.min(axis=0))
df_scores['overall_score'] = df_scores.mean(axis=1)
# df_scores['rank'] = df_scores.mean(axis=1).rank(ascending=False).astype(int)

df_all = pd.concat([df_scores, df_res], axis=1)
df_all = df_all.fillna(0)
df_all.index.name = 'method_name' 
df_all = df_all.reset_index()

df_all 


,method_name,S1,S2,static-theta-0.0,static-theta-0.5,static-theta-1.0,overall_score,Duration (hour),Peak memory (GB)
0,collectri,0.000000,0.000000,0.162018,0.220172,0.225904,0.121619,0.000000,0.000000
1,negative_control,0.000000,0.000000,0.082188,0.170247,0.224394,0.095366,0.000000,0.000000
2,positive_control,1.000000,1.000000,0.842557,0.700505,0.482437,0.805100,0.000000,0.000000
3,pearson_corr,0.798299,0.840802,0.842404,0.687843,0.461208,0.726111,0.000000,0.000000
4,portia,0.400603,0.497954,0.639161,0.338981,0.275428,0.430425,2.491111,55.685230
5,ppcor,0.000000,0.000000,0.347992,0.235417,0.216236,0.159929,13.425833,64.136433
6,grnboost2,0.971633,0.780225,0.895890,0.946268,0.782102,0.875223,7.510556,7.378796
7,scenic,0.464032,0.397320,0.783895,0.745078,0.852167,0.648498,24.008611,35.954300
8,granie,0.229666,0.177568,0.000000,0.000000,0.000000,0.081447,1.012038,41.000000
9,scglue,0.190305,0.509488,0.668041,0.307502,0.284899,0.392047,11.097500,61.677879


## Summary figure

In [22]:

summary_file = "output/summary.tsv"
summary_figure = "output/summary_figure.pdf"

df_all['memory_log'] = np.log(df_all['Peak memory (GB)']+1)
df_all['memory_log'] = np.max(df_all['memory_log'])-df_all['memory_log']


df_all["duration_log"] = np.log(df_all['Duration (hour)']+1)
df_all['duration_log'] = np.max(df_all['duration_log'])-df_all['duration_log']

df_all["duration_str"] = df_all['Duration (hour)'].round(1).astype(str)
df_all['memory_str'] =  df_all['Peak memory (GB)'].round(1).astype(str)

df_all.to_csv(summary_file, sep='\t')

!Rscript ../grn_benchmark/src/summary_figure.R {summary_file} {summary_figure}

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──
✔ ggplot2 3.5.1     ✔ purrr   0.3.4
✔ tibble  3.2.1     ✔ dplyr   1.1.4
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Warning message:
`thisfile()` was deprecated in rprojroot 2.0.0.
ℹ Please use `whereami::thisfile()` instead. 
New names:
• `` -> `...1`
Rows: 13 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (1): method_name
dbl (13): ...1, S1, S2, static-theta-0.0, static-theta-0.5, static-theta-1.0...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 10 × 7
   id               id_color         name       group geom  palette options     
   <c

# Robustness analysis

In [23]:
if True:
    !sbatch scripts/sbatch/robustness_analysis.sh # !python src/robustness_analysis/script_all.py
base_dir = 'resources/results/robustness_analysis'

Submitted batch job 7765072


In [7]:
def format_robustness_results(base_dir, noise_type='net'):
    degrees = [0, 10, 20, 50, 100]
    reg1_metric = 'S1'
    reg2_metric = 'static-theta-0.5'
    for i, degree in enumerate(degrees):
        df = pd.read_csv(f'{base_dir}/{noise_type}-{degree}-scores.csv',index_col=0)
        df_reg1 = df.loc[:, [reg1_metric]].rename(columns={reg1_metric:degree})
        df_reg2 = df.loc[:, [reg2_metric]].rename(columns={reg2_metric:degree})
        
        if i == 0:
            reg1_scores = df_reg1
            reg2_scores = df_reg2
        else:
            reg1_scores = pd.concat([reg1_scores, df_reg1], axis=1)
            reg2_scores = pd.concat([reg2_scores, df_reg2], axis=1)
        
    reg1_scores = reg1_scores.T
    reg2_scores = reg2_scores.T
    return reg1_scores, reg2_scores

## Permute net

In [9]:
# net 
noise_type = 'net'
reg1_scores, reg2_scores = format_robustness_results(base_dir, noise_type=noise_type)

In [10]:
reg1_scores.style.background_gradient()

,collectri,negative_control,positive_control,pearson_corr,portia,ppcor,grnboost2,scenic,granie,scglue,celloracle,figr,scenicplus
0,-0.052885,-0.038053,0.285482,0.227900,0.114365,-0.009030,0.277384,0.132838,0.065566,0.054329,0.177773,0.097069,0.275561
10,-0.063194,-0.037703,0.258300,0.201382,0.094161,-0.016007,0.246858,0.117373,0.052529,0.046615,0.159076,0.085768,0.259247
20,-0.068023,-0.038826,0.232850,0.188608,0.076579,-0.025485,0.213148,0.103506,0.043782,0.037751,0.145942,0.079237,0.243952
50,-0.083635,-0.035601,0.180567,0.147512,0.026785,-0.041553,0.147266,0.060212,nan,nan,nan,nan,nan
100,-0.084617,-0.038591,-0.009365,-0.045844,-0.052908,-0.064681,-0.057610,-0.016075,-0.004035,-0.045894,-0.019244,-0.013468,-0.006252


In [11]:
reg2_scores.style.background_gradient()

,collectri,negative_control,positive_control,pearson_corr,portia,ppcor,grnboost2,scenic,granie,scglue,celloracle,figr,scenicplus
0,0.314904,0.298416,0.438187,0.434587,0.337533,0.316130,0.501373,0.447392,0.268367,0.335675,0.467258,0.354542,0.514954
10,0.310607,0.300799,0.433080,0.427072,0.337533,0.314340,0.492975,0.439982,0.263060,0.334209,0.453522,0.356273,0.506758
20,0.312346,0.298023,0.428219,0.416558,0.334752,0.307381,0.481193,0.427551,0.268367,0.330640,0.444836,0.351414,0.498718
50,0.308445,0.300843,0.413975,0.401897,0.322256,0.300537,0.439232,0.399823,nan,nan,nan,nan,nan
100,0.295797,0.303241,0.379001,0.345088,0.291961,0.286424,0.293421,0.317192,0.318627,0.289704,0.310580,0.316354,0.321514


## Permute sign

In [12]:
noise_type = 'sign'
reg1_scores, reg2_scores = format_robustness_results(base_dir, noise_type=noise_type)

In [13]:
reg1_scores.style.background_gradient()

,collectri,negative_control,positive_control,pearson_corr,portia,ppcor,grnboost2,scenic,granie,scglue,celloracle,figr,scenicplus
0,-0.052885,-0.038053,0.285482,0.227900,0.114365,-0.009030,0.277384,0.132916,0.065566,0.054329,0.177773,0.097069,0.275561
10,-0.063294,-0.037943,0.244253,0.175927,0.068840,-0.026427,0.225111,0.097068,0.043191,0.039099,0.130693,0.066178,0.232978
20,-0.078692,-0.037331,0.195138,0.143907,0.030350,-0.041580,0.162380,0.059698,0.023835,0.021276,0.078184,0.049052,0.183118
50,-0.092569,-0.034971,-0.018533,-0.071464,-0.036026,-0.065349,-0.034382,-0.023101,-0.002446,-0.024859,-0.027821,-0.014739,-0.006185
100,-0.052885,-0.038053,0.285482,0.227900,0.114365,-0.009030,0.277384,0.132916,0.065566,0.054329,0.177773,0.097069,0.275561


In [14]:
reg2_scores.style.background_gradient()

,collectri,negative_control,positive_control,pearson_corr,portia,ppcor,grnboost2,scenic,granie,scglue,celloracle,figr,scenicplus
0,0.314904,0.298416,0.438187,0.434587,0.343306,0.316130,0.501373,0.447392,0.257013,0.335675,0.467258,0.354542,0.514954
10,0.315233,0.298546,0.438187,0.434587,0.343306,0.316130,0.501373,0.447392,0.257063,0.335675,0.467258,0.354542,0.514954
20,0.315206,0.298645,0.438187,0.434587,0.343306,0.316130,0.501373,0.447392,0.256943,0.335675,0.467258,0.354542,0.514954
50,0.315445,0.299033,0.438187,0.434587,0.343306,0.316130,0.501373,0.447392,0.256891,0.335675,0.467258,0.354542,0.514954
100,0.314412,0.298592,0.438187,0.434587,0.343306,0.316130,0.501373,0.447392,0.257129,0.335675,0.467258,0.354542,0.514954


## Permute weight

In [15]:
noise_type = 'weight'
reg1_scores, reg2_scores = format_robustness_results(base_dir, noise_type=noise_type)

FileNotFoundError: [Errno 2] No such file or directory: 'resources/results/robustness_analysis/weight-20-scores.csv'

# repo

In [26]:
# if par['metacell']:
#     print('metacell')
#     def create_meta_cells(df, n_cells=15):
#         meta_x = []
#         for i in range(0, df.shape[0], n_cells):
#             meta_x.append(df.iloc[i:i+n_cells, :].sum(axis=0).values)
#         df = pd.DataFrame(meta_x, columns=df.columns)
#         return df
            
#     adata_df = pd.DataFrame(multiomics_rna.X.todense(), columns=multiomics_rna.var_names)
#     adata_df['cell_type'] = multiomics_rna.obs['cell_type'].values
#     adata_df['donor_id'] = multiomics_rna.obs['donor_id'].values
#     df = adata_df.groupby(['cell_type','donor_id']).apply(lambda df: create_meta_cells(df))
#     X = df.values
#     var = pd.DataFrame(index=df.columns)
#     obs = df.reset_index()[['cell_type','donor_id']]
#     multiomics_rna = ad.AnnData(X=X, obs=obs, var=var)